A smart city power department wants to predict tomorrow’s electricity demand (in MW) using the last 7 days of hourly consumption data.
This helps to:
Avoid power shortages ⚡
Optimize power generation scheduling
Reduce operational costs
🔹 Sample Dataset (Simplified – Daily Average)

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv("electricity_daily_avg.csv").sort_values("Day").reset_index(drop=True)
target_col = "Electricity Demand (MW)"  

series = df[target_col].to_numpy(dtype=np.float32).reshape(-1, 1)

N = len(series)
train_size = int(N * 0.70)
val_size = int(N * 0.15)

train = series[:train_size]
val   = series[train_size:train_size + val_size]
test  = series[train_size + val_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
val_scaled   = scaler.transform(val)
test_scaled  = scaler.transform(test)

def make_sequences(data, look_back):
    data = np.asarray(data)
    if data.ndim == 1:
        data = data.reshape(-1, 1)

    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, :])   
        y.append(data[i+look_back, 0])     

    X = np.array(X, dtype=np.float32)      
    y = np.array(y, dtype=np.float32)        
    return X, y

LOOK_BACK = 7
X_train, y_train = make_sequences(train_scaled, LOOK_BACK)
X_val, y_val     = make_sequences(val_scaled, LOOK_BACK)
X_test, y_test   = make_sequences(test_scaled, LOOK_BACK)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val  :", X_val.shape,   "y_val  :", y_val.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)

tf.keras.utils.set_random_seed(42)


model = Sequential([
    GRU(64, return_sequences=True, input_shape=(LOOK_BACK, 1)),
    Dropout(0.2),

    LSTM(32, return_sequences=False),   # <- LSTM added here
    Dropout(0.2),

    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=150,
    batch_size=16,
    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)],
    verbose=1
)


full_scaled = scaler.transform(series)  
last_seq = full_scaled[-LOOK_BACK:].reshape(1, LOOK_BACK, 1)
tomorrow_scaled = model.predict(last_seq)
tomorrow_mw = scaler.inverse_transform(tomorrow_scaled)[0, 0]

print(f"Predicted demand for tomorrow: {tomorrow_mw:.2f} MW")

X_train: (77, 7, 1) y_train: (77,)
X_val  : (11, 7, 1) y_val  : (11,)
X_test : (11, 7, 1) y_test : (11,)
Epoch 1/150


d:\Training\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 147ms/step - loss: 0.3399 - val_loss: 0.2942
Epoch 2/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1477 - val_loss: 0.1146
Epoch 3/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0979 - val_loss: 0.1418
Epoch 4/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1051 - val_loss: 0.1149
Epoch 5/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0791 - val_loss: 0.1168
Epoch 6/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0867 - val_loss: 0.1247
Epoch 7/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0773 - val_loss: 0.1194
Epoch 8/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0958 - val_loss: 0.1127
Epoch 9/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0951 - val_loss: 0.1099
Epoch 10/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0869 - val_loss: 0.1097
Epoch 11/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0782 - val_loss: 0.1102
Epoch 12/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0825 - val_loss: 0.1107
